# ETL de ventas de SUMUP

Extraeremos las ventas a través de la API de SUMUP y añadiremos esa información a las bases de datos de ventas Nikarit en Bigquery y en Firebase. <br>
¿Qué es SUMUP? - Sumup es la aplicación que usamos para registrar ventas cuando estamos en mercadillos. Registramos tanto ventas manuales como ventas a través del datáfono de SUMUP <br>
<br>
La idea es integrar estas ventas con las ventas online en una misma base de datos

In [1]:
# Imports
import pandas as pd

import requests
import yaml
import json
import os
import firebase_admin
import subprocess
import googleapiclient.discovery
import string
import numpy as np

from time import sleep

from google.oauth2 import service_account
from google.cloud import bigquery


from datetime import datetime,timedelta

#Firbease (pip install firebase_admin)
from firebase_admin import credentials
from firebase_admin import firestore

## google cloud
import google.auth.transport.requests as grequests
from google.oauth2.id_token import fetch_id_token

from dotenv import load_dotenv
 
load_dotenv(dotenv_path='/Users/daniel/OAN/credentials/miong/.env')

shop_url = os.environ['SHOPIFY_ACCESS_URL']

cloudCreds = os.environ['CLOUD_CREDS']


## GLOBAL VARIABLES

In [2]:
CREATEDBYTYPE_SYSTEM = "system"
CREATEDBYTYPE_USER = "user"
CREATEDBYTYPE_SUMUP = "sumup"

In [3]:
OAN_account = "5Tv2u4n8BReebmKUNIuN"
DONANTES_ID = os.environ['DONANTES_ID']
creation_date = datetime.today().isoformat()[:-3]+'Z'

In [4]:
DANIEL_USER_ID = "z5m936GA0t3vHM28QKhR"
BOT_USER_ID = "oXJJEfAEPxFYtdJ2pnaU"
INVENTORY_DANIEL = "ojyJQVfxUeu33AzOehNw"

In [5]:
CLIENTGENDERFEMALE = "Female"
CLIENTGENDERMALE = "Male"
CLIENTGENDERUNKNOWN = "Unknown"

In [6]:
GATEWAYCASH = "cash"
GATEWAYCARD = "card"
GATEWAYPAYPAL = "paypal"
GATEWAYSHOPIFY = "shopify_payments"

In [7]:
URL_onCreateSaleAPI = "https://us-central1-oan-miong.cloudfunctions.net/onCreateSaleAPI"

In [8]:
#para conseguir todas las transacciones
URL_transHist = "https://api.sumup.com/v0.1/me/transactions/history"

#para conseguir más información de cada una de las transacciones
URL_transactions = "https://api.sumup.com/v0.1/me/transactions"

# ETL MAPPING

### PRODUCTOS

In [10]:
sql = """
    SELECT *
    FROM `oan-miong.firestore_export.shopProductTypesView`
    WHERE account = "5Tv2u4n8BReebmKUNIuN"
"""

# Run a Standard SQL query using the environment's default project
df_productos = pd.read_gbq(sql, dialect='standard')

In [11]:
prods_name = df_productos['name'].to_list()

In [12]:
d_prods_transProd = dict(zip(prods_name,["" for a in prods_name]))
d_prods_transId = dict(zip(prods_name,["" for a in prods_name]))

In [13]:
miOngProducts = df_productos['name'].to_list()
miOngProducts.sort()

In [14]:
d_prods_transProd = {'03 - Bálsamo Labial': 'Balsamo labial',
                     '02 - Manteca de karité pura - 100 ml': 'Bote manteca pura 100ml',
                     '05 - Bolsitas de Tela Africana': 'Bolsa africana',
                     '06 - Donaciones (1€)': 'Donacion',
                     '04 - Crema de Manos': 'Crema de manos',
                     '01 - Pack "La más hidratada del lugar"': 'Pack la mas hidratada',
                     'Jaboncito': 'BIO - Jabon ECO de 20g - Natura',
                     'Estrellita Navidad': 'Estrellas ',
                     'Crema Corporal BIO': 'BIO - Crema Corporal ECO 200 ml - Natura',
                     'Crema de Manos BIO': 'BIO - Crema de manos ECO 50ml - NAAY',
                     'Jabon Sólido ECO 135g': 'BIO - Jabon Pastilla Olor neutro ECO 130 g - Natura',
                     'Batic Africano': 'Donacion',
                     'Balsamo Labial BIO': 'BIO - Balsamo labial Vegano ECO 15ml - NAAY',
                     'Manteca de karité BIO': 'BIO - Manteca de karite ECO 100ml',
                     'Importe personalizado': 'Donacion',
                     'Tote Bag': 'Tote bag',
                     'Cartera': 'Donacion',
                     'Neceser': 'Donacion',
                     'Pack Bio': 'BIO - Pack la mas hidratada',
                     'Llavero Africano': 'Donacion',
                     'Custom amount': 'Donacion',
                     'Pack Jabon y Crema Corporal': 'Pack 100% Benin',
                     'Pack Navideño': 'Pack navideño 2023'}

In [15]:
for prod in d_prods_transProd:
    if d_prods_transProd[prod] != 'Donacion':
        d_prods_transId[prod] = df_productos[df_productos['name']==d_prods_transProd[prod]]['id'].iloc[0]
    else:
        d_prods_transId[prod] = 'Donacion'

### MERCADILLOS

In [16]:
MERCADILLOZIELO = "Lj5kN2g1MKCKPnOvCntZ"
MERCADILLOLASROZAS = "UI8ZyG7ttkWt1UO7H5aV"
MERCADILLOMAGIAPORBENIN = "8Vj96dZ5OHL7MMGiXZ9I"
MERCADILLOOTROS = "ueILrWpu2TYsLElIBeFW"

In [17]:
d_fechas_mercadillos = {'2021-11-27': '',
                         '2022-04-22': MERCADILLOMAGIAPORBENIN,
                         '2022-04-23': MERCADILLOMAGIAPORBENIN,
                         '2022-04-24': MERCADILLOMAGIAPORBENIN,
                         '2022-12-17': MERCADILLOZIELO,
                         '2023-03-04': MERCADILLOLASROZAS,
                         '2023-04-02': MERCADILLOLASROZAS,
                         '2023-05-05': MERCADILLOOTROS,
                         '2023-05-06': MERCADILLOLASROZAS,
                         '2023-05-19': MERCADILLOMAGIAPORBENIN,
                         '2023-05-20': MERCADILLOMAGIAPORBENIN,
                         '2023-05-21': MERCADILLOMAGIAPORBENIN,
                         '2023-06-01': MERCADILLOLASROZAS,
                         '2023-06-03': MERCADILLOLASROZAS,
                         '2023-06-17': MERCADILLOOTROS,
                         '2023-07-07': '',
                         '2023-10-07': MERCADILLOLASROZAS,
                         '2023-12-01': '',
                         '2023-12-02': MERCADILLOZIELO,
                         '2023-12-16': '',
                         '2023-12-19': ''}

# FUNCTIONS

In [18]:
def createSaleDict(code,dataSale):
    randomId = db.collection('tmp').document().id
    creation_date = datetime.today().isoformat()[:-3]+'Z'
    sale = {
            "context":{
                "id":randomId,
                "account":OAN_account,
                "createdAt":dataSale["createdAt"],
                "createdBy":BOT_USER_ID,
                "createdByType":CREATEDBYTYPE_SUMUP,
                "lastUpdateAt":creation_date,
                "lastUpdateBy":""
            },
            "code": code,
            "accountingItem" : "",
            "inventory": INVENTORY_DANIEL,
            "customer":dataSale["customer"],
            "clientEmail" : "",
            "clientName" : "",
            "clientLastName" : "",
            "clientGender" : CLIENTGENDERUNKNOWN,
            "clientPhone" : "",
            "clientAddress" : "",

            "closeDate" : dataSale["closeDate"],

            "discountPrice" : 0,
            "totalPrice" : dataSale["totalPrice"],
            "subtotalPrice" : dataSale["subtotalPrice"],
            "totalTax" : dataSale["totalTax"],
            "taxesIncluded" : True,
            "donation":dataSale["donation"],
            "buyerAcceptsMarketing" : False,
            "orderId":"",
            "orderStatusUrl" : "",
            "gateway" : dataSale["gateway"],
            "latitude" : dataSale["latitude"],
            "longitude" : dataSale["longitude"],
            "country" : "Spain",
            "province" : "",
            "city" : "",
            "zip" : "",
            "arePricesWithoutTaxes":False,
            "products":dataSale["productos"] #[{"productType" : d_prods_transId[a['name']], "quantity":a['quantity'], "unitPrice":a['price']}]

        }

    data = {
            "accountId":OAN_account,
            "userId":BOT_USER_ID,
            "sale":sale
            }

    SaleItem = {
        "data": data
    }
    return SaleItem

## ACCESS FIREBASE

In [19]:
# Use a service account
cred = credentials.Certificate(cloudCreds)
firebase_admin.initialize_app(cred)

In [20]:
db = firestore.client()

## ACCESS TO FUNCTION CLOUD

In [21]:
url = URL_onCreateSaleAPI
auth_req = grequests.Request()
id_token = fetch_id_token(auth_req, url)
headers = {"Authorization":"Bearer {}".format(id_token),
           'Content-type': 'application/json'}

In [22]:
#r = grequests.requests.post(url, json=SaleItemEx,headers=headers)

## NOS CONECTAMOS A SUMUP

In [23]:
#cogemos la ruta del archivo con las credenciales de sumup
sumup_file = os.environ['SUMUP']
#abrimos el archivo y lo cargamos en un diccionario
creds_sumup = json.load(open(sumup_file,'r'))
#endpoints de autentificación
authorize_url = creds_sumup["authorize_url"]
token_url = creds_sumup["token_url"]
client_id = creds_sumup["client_id"]
client_secret = creds_sumup["client_secret"]
callback_uri = creds_sumup["redirect_uris"]
scopes = ['transactions.history','user.profile','user.profile_readonly']

info sobre las transacciones - # https://developer.sumup.com/rest-api/#tag/Transactions

### AUTENTIFICACION EN SUMUP PARA OBTENER CÓDIGO
Tal y como describe en documentación, la api de sumup, para poder recibir un token de acceso necesita <br>
que te autentiques como usuario para poder darte un token de acceso que puede permitirte <br>
acceder a la información de tu usuario

In [24]:
authorization_redirect_url = (authorize_url +
                              '?response_type=code&client_id=' +
                              client_id +
                              '&redirect_uri=' + 
                              callback_uri[0] +
                              '&scope='+scopes[0])
print(authorization_redirect_url)

https://api.sumup.com/authorize?response_type=code&client_id=cc_classic_zrN4KqPtdpjz0nLtCEiP6tXBN3E5M&redirect_uri=http://oan.miong.es&scope=transactions.history


In [26]:
#Añadimos el código que nos ha generado
code = input('enter code:\n')

enter code:
wNbFC3AePnc6TqSBs5oM9wqBzGvewAPjsRmw


In [27]:
#cojemos el token de acceso
data = {'grant_type': 'authorization_code',
        'code': code,
        'redirect_uri': callback_uri}
access_token_response = requests.post(token_url,
                                      data=data,
                                      allow_redirects=False,
                                      auth=(client_id, client_secret))

tokens = json.loads(access_token_response.text)
access_token = tokens['access_token']
api_call_headers = {'Authorization': 'Bearer ' + access_token}

---- ya estamos autenticados

## COGEMOS LISTADO DE TRANSACCIONES

In [28]:
sumupSalesFirebase = db.collection('shopSales'
                             ).where(u'context.account', u'==',OAN_account
                                    ).where("context.createdByType", "==", "sumup"
                                          ).stream()

In [29]:
l_sumupsales = []
for sale in sumupSalesFirebase:
    #print(payout)
    l_sumupsales.append(sale.to_dict())

In [30]:
dfFirebase = pd.DataFrame(l_sumupsales)

In [48]:
dfFirebase.shape

(1296, 31)

In [38]:
dfFirebaseReciente = dfFirebase[dfFirebase['context'].apply(lambda x: x['lastUpdateAt'] < '2023-12-01')]

In [47]:
dfFirebaseReciente.shape

(774, 31)

In [49]:
maxFirebaseDate = dfFirebaseReciente.closeDate.max()[:-2]

In [50]:
maxFirebaseDate

'2021-11-27T19:28:29.76'

In [51]:
params = {
    "limit":1000,
    "oldest_time":maxFirebaseDate,
}
api_call_response = requests.get(URL_transHist, 
                                 headers=api_call_headers, 
                                 params=params)

In [52]:
'todo correcto' if api_call_response.status_code == 200 else 'La liaste'

'todo correcto'

In [53]:
ventasSumupJson = api_call_response.json()

In [54]:
ventasSumup = ventasSumupJson['items']

In [55]:
dfVentasSumupAll = pd.DataFrame(ventasSumup)

In [56]:
dfVentasSumupAll['amount'].sum()

9557.66

In [57]:
dfVentasSumupAll.shape

(576, 20)

In [58]:
dfVentasSumupAll.columns

Index(['id', 'transaction_id', 'user', 'type', 'status', 'timestamp',
       'currency', 'amount', 'transaction_code', 'product_summary',
       'installments_count', 'payment_type', 'entry_mode', 'card_type',
       'payouts_total', 'payouts_received', 'payout_date', 'payout_plan',
       'payout_type', 'refunded_amount'],
      dtype='object')

In [59]:
dfVentasSumupAll["status"].value_counts()

SUCCESSFUL    522
FAILED         54
Name: status, dtype: int64

In [60]:
dfVentasSumup = dfVentasSumupAll[dfVentasSumupAll["status"] == "SUCCESSFUL"].copy()

In [76]:
dfVentasSumup['amount'].sum()

8638.65

### Tratamiento de los que tiene product_summary

In [77]:
dfVentasSumupConProduct = dfVentasSumup[~dfVentasSumup["product_summary"].isnull()].reset_index(drop=True).copy()

In [63]:
dfVentasSumupConProduct['quantity'] = dfVentasSumupConProduct["product_summary"
                                                             ].apply(lambda x: str(x).split(" x ")) \
                                                                .apply(lambda x: int(x[0]) if len(x)>1
                                                                    else 1)

In [64]:
dfVentasSumupConProduct['product'] = dfVentasSumupConProduct["product_summary"
                                                             ].apply(lambda x: str(x).split(" x ")) \
                                                                .apply(lambda x: x[1] if len(x)>1
                                                                                else x[0])

In [66]:
listaConProd = list(dfVentasSumupConProduct['product'].unique())

In [67]:
listaConProdAll = dict(zip(listaConProd,
                           [d_prods_transId[a] 
                            if a in d_prods_transProd.keys() 
                            else 'Donacion' 
                            for a in listaConProd]
                          )
                      )


In [68]:
dfVentasSumupConProduct['productPrice'] = dfVentasSumupConProduct['amount']/dfVentasSumupConProduct['quantity']

In [69]:
def matcheoProdsCon(x):
    prods = []
    donacion = 0
    for a in x:
        if listaConProdAll[x['product']] != 'Donacion':
            prods = [{'productType':listaConProdAll[x['product']] ,
                                          'quantity':x['quantity'],
                                          'unitPrice': x['productPrice']}]
            
        else:
            donacion = float(x['amount'])
    return pd.Series([donacion, prods])

In [70]:
dfVentasSumupConProduct[['donation','productos']] = dfVentasSumupConProduct.apply(lambda x: 
                                                                                  matcheoProdsCon(x), axis=1)


### COGEMOS LA INFORMACION DE LOS PRODUCTOS QUE SE VENDIERON EN CADA VENTA

In [515]:
transactions = dfVentasSumupConProduct['transaction_id'].to_list()

In [526]:
def get_info_trans(x):
    products = []
    lat = 0
    lon = 0
    transaction=''
    transaction_id=x
    params = {
        "id":transaction_id
    }
    api_call_response = requests.get(URL_transactions, 
                                     headers=api_call_headers, 
                                     params=params)
    status = api_call_response.status_code
    if status != 200:
        raise('error in {} - {}'.format(x,status))
    
    else:
        transaction = api_call_response.json()
        products = transaction['products']
        lat = transaction['lat']
        lon = transaction['lon']
    sleep(2.5)
    print(transactions.index(x))
    return pd.Series([products,lat,lon])

In [527]:
dfVentasSumupConProduct2.shape

(173, 22)

In [ ]:
dfVentasSumupConProduct2[['products','lat','lon']] = dfVentasSumupConProduct2['transaction_id'
                                                                       ].apply(lambda x: get_info_trans(x))


In [190]:
#df_ventas_sumup_original = df_ventas_sumupNan.copy()

In [342]:
df_ventas_sumup = df_ventas_sumupNan.copy()

In [343]:
df_ventas_sumup.shape

(159, 26)

In [344]:
df_ventas_sumup = df_ventas_sumup[df_ventas_sumup['status']=="SUCCESSFUL"].reset_index()

In [345]:
df_ventas_sumup.shape

(149, 27)

### TRANSFORMAMOS LA INFORMACION PARA QUE ESTE IGUAL QUE EN LA BBDD

#### COGEMOS listado de productos únicos 

In [346]:
l_all_prods = df_ventas_sumup['products'].to_list()

In [347]:
l_prods = []
for prods in l_all_prods:
    l_prods = l_prods+prods

In [348]:
df_prods = pd.DataFrame(l_prods)

In [349]:
df_prods[['name','price','vat_rate','price_with_vat']].drop_duplicates(subset=['name'])

,name,price,vat_rate,price_with_vat
0,03 - Bálsamo Labial,3.31,0.21,4.0
1,02 - Manteca de karité pura - 100 ml,8.26,0.21,10.0
4,05 - Bolsitas de Tela Africana,4.13,0.21,5.0
5,06 - Donaciones (1€),1.00,0.00,1.0
10,04 - Crema de Manos,4.96,0.21,6.0
27,"01 - Pack ""La más hidratada del lugar""",16.53,0.21,20.0
30,Jaboncito,1.00,0.00,1.0
31,Estrellita Navidad,1.00,0.00,1.0
37,Crema Corporal BIO,14.88,0.21,18.0
39,Crema de Manos BIO,6.61,0.21,8.0


In [350]:
prods_name = df_prods['name'].unique()

In [351]:
prods_name.sort()

### los productos que estan en blanco, es porque no se dieron de alta en la venta

## GESTIONAR EL MATCHEO

## GET DATA WANTED

In [388]:
def matcheoProds(x):
    prods = []
    donacion = 0
    for a in x:
        if "price_with_vat" in a.keys() and d_prods_transId[a['name']] != 'Donacion':
            prods = prods + [{"productType" : d_prods_transId[a['name']],
                                              "quantity":a['quantity'],
                                              "unitPrice":a['price_with_vat']
                                             }]
        elif d_prods_transId[a['name']] != 'Donacion':
            prods = prods + [{"productType" : d_prods_transId[a['name']],
                                              "quantity":a['quantity'],
                                              "unitPrice":a['price']
                                             }]
        elif d_prods_transId[a['name']] == 'Donacion' and "total_with_vat" in a.keys():
            donacion = donacion+float(a['total_with_vat'])
        elif d_prods_transId[a['name']] == 'Donacion':
            donacion = donacion+float(a['total_price'])
    return pd.Series([donacion, prods])

In [985]:
df_ventas_sumup = dfVentasSumupConProduct.copy()

In [987]:
df_ventas_sumup.columns

Index(['id', 'transaction_id', 'user', 'type', 'status', 'timestamp',
       'currency', 'amount', 'transaction_code', 'product_summary',
       'installments_count', 'payment_type', 'entry_mode', 'card_type',
       'payouts_total', 'payouts_received', 'payout_date', 'payout_plan',
       'payout_type', 'refunded_amount', 'quantity', 'product', 'productPrice',
       'donation', 'productos'],
      dtype='object')

In [990]:
df_ventas_sumup[['donation','productos']] = df_ventas_sumup.apply(lambda x: matcheoProdsCon(x),axis=1)

In [577]:
df_ventas_sumup['createdAt'] = df_ventas_sumup.timestamp
df_ventas_sumup['closeDate'] = df_ventas_sumup.timestamp

trad_paytype = {'POS':'card','CASH':'cash','ECOM':'card'}

df_ventas_sumup['gateway'] = df_ventas_sumup['payment_type'].apply(lambda x: trad_paytype[x]) 


In [579]:
df_ventas_sumup.columns

Index(['id', 'transaction_id', 'user', 'type', 'status', 'timestamp',
       'currency', 'amount', 'transaction_code', 'product_summary',
       'installments_count', 'payment_type', 'entry_mode', 'card_type',
       'payouts_total', 'payouts_received', 'payout_date', 'payout_plan',
       'payout_type', 'refunded_amount', 'quantity', 'product', 'productPrice',
       'donation', 'productos', 'createdAt', 'closeDate', 'gateway'],
      dtype='object')

In [1081]:
bolla=0
if bolla == 1:
    df_ventas_sumup['totalTax'] = df_ventas_sumup['products'].apply(lambda x: round(sum([a['vat_amount'] 
                                                                                    for a in x
                                                                                   if 'vat_amount' in a.keys()]
                                                                                       ),2
                                                                                   )
                                                                    )
                                                    
else:                                                               
    df_ventas_sumup['totalTax'] = (df_ventas_sumup['amount']-df_ventas_sumup['donation'])*(1-1/1.21)
    df_ventas_sumup['lat'] = None
    df_ventas_sumup['lon'] = None

df_ventas_sumup['subtotalPrice'] = df_ventas_sumup['amount']-df_ventas_sumup['donation']


In [1085]:
df_ventas_sumup['donation'].sum()

42262.5

In [1086]:
df_ventas_sumup['amount'].sum()

4703.53

In [1083]:
df_ventas_sumup['totalTax'].sum()

-6518.498925619834

In [589]:
df_ventas_sumup['latitude'] = df_ventas_sumup['lat']
df_ventas_sumup['longitude'] = df_ventas_sumup['lon']
df_ventas_sumup['totalPrice'] = df_ventas_sumup['amount']

In [596]:
df_ventas_sumup['customer'] = pd.to_datetime(df_ventas_sumup['closeDate']
                                            ).dt.date.apply(lambda x: d_fechas_mercadillos[str(x)]
                                                           )

In [449]:
colsToMatch = []

In [977]:
df_ventas_sumup.shape

(373, 36)

In [1087]:
dfVentasfinal.shape

(522, 12)

In [635]:
cols = ['latitude','longitude','totalPrice','customer','subtotalPrice',
                                           'totalTax','gateway','createdAt','closeDate','donation','productos']
dfVentasfinal = pd.concat([df_ventas_sumup[cols],
                           df_ventas_sumupNoProd[cols]
                          ],
                          axis=0).reset_index()

In [1095]:
def matcheoProdsDonacion(x):
    donacion = 0
    if len(x['productos'])==0:
        donacion = x['totalPrice']
    else:
        donacion = x['totalPrice']-np.array([a['quantity']*a['unitPrice'] for a in x['productos']]).sum()
    return pd.Series(donacion)

In [1099]:
dfVentasfinal['subtotalPrice'] = dfVentasfinal['totalPrice']-dfVentasfinal['donation'] 

In [638]:
alldata = dfVentasfinal.T.to_dict()

In [1125]:
dfVentasfinal.to_csv('dataFinal.csv',index=False)

In [1126]:
dfVentasfinalcsv = pd.read_csv('dataFinal.csv',index_col='index')

In [ ]:
alldata = dfVentasfinalcsv.T.to_dict()

In [ ]:
db.collection('info').document(OAN_account+"-shopSales").

In [640]:
code = db.collection('info').document(OAN_account+"-shopSales").get().to_dict()['counter']
for i in alldata:
    dataSale = alldata[i]
    dataSaleJson = createSaleDict(code,dataSale)
    code = code+1
    r = grequests.requests.post(url, json=dataSaleJson,headers=headers)

In [641]:
#actualizar el counter
db.collection('info').document(OAN_account+'-shopSales').update({'counter':code})

update_time {
  seconds: 1703042979
  nanos: 751534000
}

### ADD TO GBQ

In [684]:
dfVentasfinal.shape

(522, 12)

In [74]:
querySales = '''
            SELECT *
            FROM `oan-miong.firestore_export.shopSales`
            '''
dfGbqSales = pd.read_gbq(query=querySales)

In [75]:
dfGbqSales.shape

(3379, 37)

In [69]:
shopSalesQuery = db.collection('shopSales').stream()

In [70]:
shopSalesDocs = []
for shoSale in shopSalesQuery:
    shopSalesDocs.append(shoSale.to_dict())

In [71]:
dfFirebase = pd.DataFrame(shopSalesDocs)

In [73]:
dfFirebase.shape

(3380, 33)

In [76]:
colsToAdd = dfFirebase['context'].iloc[1000].keys()
for col in colsToAdd:
    dfFirebase[col] = dfFirebase['context'].apply(lambda x: x[col] if col in x.keys() else "")

In [77]:
dfFirebase['action'] = 'created'

In [78]:
dfFirebase['timestamp'] = dfFirebase['createdAt']

In [79]:
dfFirebase['closeDate'] = dfFirebase['closeDate'].fillna(dfFirebase['createdAt'])

In [80]:
colsInGbq = list(dfFirebase.columns)
for notCol in list(set(dfFirebase.columns)-set(dfGbqSales.columns)):
    colsInGbq.remove(notCol)

In [81]:
colsInGbq = dfGbqSales.columns

In [82]:
dfFirebasetoUpdate = dfFirebase[colsInGbq].copy()

In [83]:
dfFirebasetoUpdate['code'] = dfFirebasetoUpdate['code'].fillna(0).apply(lambda x: 0 if x == None else int(x))
dfFirebasetoUpdate['discountPrice'] = dfFirebasetoUpdate['discountPrice'].apply(lambda x: 0 if x == "" 
                                                                                                or x == None
                                                                                            else float(x))


In [84]:
dfGbqSales.shape

(3379, 37)

In [85]:
#dfFinal = pd.concat([dfGbqSales,dfFirebasetoUpdate])
dfFinal = dfFirebasetoUpdate

In [86]:
dfFinal['discountPrice'] = dfFinal['discountPrice'].apply(lambda x: 0 if x == "" or x == None
                                                                                            else float(x))

In [87]:
dfFinal.shape

(3380, 37)

In [88]:
dfFinal['code'] = dfFinal['code'].apply(int)

In [89]:
colsToStr = list(dfFinal.columns)
for a in ['code','totalPrice','subtotalPrice','totalTax','donation','latitude','longitude','discountPrice']:
    colsToStr.remove(a)

In [90]:
for a in colsToStr:
    dfFinal[a] = dfFinal[a].astype(str)

In [91]:
dfFinal[dfFinal['closeDate'].isnull()]

,id,action,timestamp,account,createdByType,createdBy,lastUpdateBy,createdAt,lastUpdateAt,code,...,buyerAcceptsMarketing,orderStatusUrl,gateway,latitude,longitude,country,province,city,zip,products


In [92]:
dfFinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3380 entries, 0 to 3379
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3380 non-null   object 
 1   action                 3380 non-null   object 
 2   timestamp              3380 non-null   object 
 3   account                3380 non-null   object 
 4   createdByType          3380 non-null   object 
 5   createdBy              3380 non-null   object 
 6   lastUpdateBy           3380 non-null   object 
 7   createdAt              3380 non-null   object 
 8   lastUpdateAt           3380 non-null   object 
 9   code                   3380 non-null   int64  
 10  orderId                3380 non-null   object 
 11  accountingItem         3380 non-null   object 
 12  inventory              3380 non-null   object 
 13  customer               3380 non-null   object 
 14  clientEmail            3380 non-null   object 
 15  clie

## ACTUALIZAR BIGQUERY

In [93]:
bgq_client = bigquery.Client()

In [94]:
dfFinal.to_gbq('oan-miong.firestore_export.shopSales',
                       project_id='oan-miong',
                       if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 1760.09it/s]


In [95]:
sql = """
CREATE OR REPLACE TABLE `oan-miong.firestore_export.shopSales` AS
SELECT * EXCEPT (products),
    ARRAY(
      SELECT STRUCT(

      JSON_VALUE(prod, '$.productType') AS productType,
      CAST(REPLACE(JSON_VALUE(prod, '$.quantity'),".0","") AS INT64) AS quantity,
      CAST(JSON_VALUE(prod, '$.unitPrice') AS FLOAT64) AS unitPrice
      )
      FROM UNNEST(JSON_EXTRACT_ARRAY(products, '$')) AS prod) AS products
FROM
  `oan-miong.firestore_export.shopSales`
"""

In [96]:
query_job = bgq_client.query(sql)

In [97]:
rows = query_job.result()